In [29]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [30]:
# load data
df=pd.read_csv('PenDigits_withoutdupl_norm_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,outlier
0,48.0,100.0,20.0,80.0,0.0,51.0,55.0,43.0,100.0,56.0,90.0,62.0,86.0,30.0,73.0,0.0,1
1,38.0,100.0,4.0,85.0,0.0,53.0,58.0,46.0,100.0,52.0,88.0,64.0,78.0,32.0,73.0,0.0,1
2,53.0,100.0,25.0,70.0,0.0,39.0,50.0,34.0,100.0,42.0,85.0,64.0,77.0,32.0,80.0,0.0,1
3,12.0,100.0,0.0,71.0,25.0,43.0,86.0,44.0,100.0,75.0,90.0,52.0,90.0,20.0,90.0,0.0,1
4,26.0,100.0,0.0,83.0,23.0,51.0,83.0,42.0,100.0,55.0,79.0,84.0,56.0,42.0,37.0,0.0,1


In [31]:
df.shape

(9868, 17)

In [32]:
# data to values
df = df.values

In [33]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Pendigits.json")
gc = GCForest(config)

In [34]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,16]
X = df[:,0:16]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [35]:
# number of class
len(np.unique(y_train))

2

In [36]:
           # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 00:56:25,104][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 16)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 16)],y_test.shape=(3257,)
[ 2018-04-25 00:56:25,106][cascade_classifier.fit_transform] group_dims=[16]
[ 2018-04-25 00:56:25,108][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:56:25,110][cascade_classifier.fit_transform] group_ends=[16]
[ 2018-04-25 00:56:25,111][cascade_classifier.fit_transform] X_train.shape=(6611, 16),X_test.shape=(3257, 16)
[ 2018-04-25 00:56:25,113][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 16), X_cur_test.shape=(3257, 16)
[ 2018-04-25 00:56:25,809][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-25 00:56:26,632][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-25 00:56:27,449][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

In [37]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:56:33,288][cascade_classifier.transform] X_groups_test.shape=[(3257, 16)]
[ 2018-04-25 00:56:33,291][cascade_classifier.transform] group_dims=[16]
[ 2018-04-25 00:56:33,293][cascade_classifier.transform] X_test.shape=(3257, 16)
[ 2018-04-25 00:56:33,295][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 16)


Test Accuracy of ExtraTrees = 99.877188 %


In [38]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 00:56:34,863][cascade_classifier.transform] X_groups_test.shape=[(3257, 16)]
[ 2018-04-25 00:56:34,865][cascade_classifier.transform] group_dims=[16]
[ 2018-04-25 00:56:34,867][cascade_classifier.transform] X_test.shape=(3257, 16)
[ 2018-04-25 00:56:34,869][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 16)


In [39]:
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 00:56:36,526][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 18)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 18)],y_test.shape=(3257,)
[ 2018-04-25 00:56:36,529][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-04-25 00:56:36,530][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:56:36,532][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-04-25 00:56:36,534][cascade_classifier.fit_transform] X_train.shape=(6611, 18),X_test.shape=(3257, 18)
[ 2018-04-25 00:56:36,536][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 18), X_cur_test.shape=(3257, 18)
[ 2018-04-25 00:56:37,232][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-25 00:56:38,062][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-25 00:56:38,838][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

In [40]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:56:44,701][cascade_classifier.transform] X_groups_test.shape=[(3257, 18)]
[ 2018-04-25 00:56:44,704][cascade_classifier.transform] group_dims=[18]
[ 2018-04-25 00:56:44,705][cascade_classifier.transform] X_test.shape=(3257, 18)
[ 2018-04-25 00:56:44,707][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 18)


Test Accuracy of ExtraTrees = 99.938594 %


In [41]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [42]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 00:56:46,325][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 18)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 18)],y_test.shape=(3257,)
[ 2018-04-25 00:56:46,327][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-04-25 00:56:46,329][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:56:46,331][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-04-25 00:56:46,332][cascade_classifier.fit_transform] X_train.shape=(6611, 18),X_test.shape=(3257, 18)
[ 2018-04-25 00:56:46,334][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 18), X_cur_test.shape=(3257, 18)
[ 2018-04-25 00:56:46,978][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-25 00:56:47,804][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-25 00:56:48,581][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

('Test Accuracy of ExtraTrees = 99.969297 %', 0, 'fois')


[ 2018-04-25 00:56:56,357][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-25 00:56:57,135][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-25 00:56:57,936][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-25 00:56:58,729][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-25 00:56:59,515][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-25 00:57:00,362][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-25 00:57:01,143][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-25 00:57:01,887][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

('Test Accuracy of ExtraTrees = 99.969297 %', 1, 'fois')


[ 2018-04-25 00:57:05,923][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-25 00:57:06,745][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-25 00:57:07,533][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-25 00:57:08,318][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-25 00:57:09,091][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-25 00:57:09,910][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-25 00:57:10,725][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-25 00:57:11,563][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

...

('Test Accuracy of ExtraTrees = 99.969297 %', 17, 'fois')


[ 2018-04-25 00:59:37,666][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-25 00:59:38,440][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-25 00:59:39,214][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-25 00:59:39,978][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-25 00:59:40,776][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-25 00:59:41,566][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-25 00:59:42,367][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-25 00:59:43,096][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

('Test Accuracy of ExtraTrees = 99.969297 %', 18, 'fois')


[ 2018-04-25 00:59:46,838][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-25 00:59:47,569][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-25 00:59:48,449][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-25 00:59:49,221][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-25 00:59:49,972][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-25 00:59:50,717][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-25 00:59:51,453][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-25 00:59:52,223][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

('Test Accuracy of ExtraTrees = 99.969297 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 99.969297 %', 0, 'fois')